In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"himeshmaniyar","key":"7eddbac61891bad3a95f8f1544454d15"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d fedesoriano/heart-failure-prediction

  0% 0.00/8.56k [00:00<?, ?B/s]
100% 8.56k/8.56k [00:00<00:00, 6.99MB/s]


In [4]:
!unzip '/content/heart-failure-prediction.zip'

Archive:  /content/heart-failure-prediction.zip
  inflating: heart.csv               


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
dataframe=pd.read_csv('./heart.csv')

In [3]:
dataframe.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [5]:
dataframe.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [6]:
dataframe["HeartDisease"].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [7]:
for x in range(len(dataframe)):
  dataframe.at[x,"Sex"]=1 if dataframe.at[x,"Sex"]=="M" else 0

In [8]:
dataframe["ChestPainType"].unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [9]:
for x in range(len(dataframe)):
  if dataframe.at[x,"ChestPainType"]=="ATA":
    dataframe.at[x,"ChestPainType"]=0
  elif dataframe.at[x,"ChestPainType"]=="NAP":
    dataframe.at[x,"ChestPainType"]=1
  elif dataframe.at[x,"ChestPainType"]=="ASY":
    dataframe.at[x,"ChestPainType"]=2
  elif dataframe.at[x,"ChestPainType"]=="TA":
    dataframe.at[x,"ChestPainType"]=3

In [10]:
dataframe["RestingECG"].unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [11]:
for x in range(len(dataframe)):
  if dataframe.at[x,"RestingECG"]=="Normal":
    dataframe.at[x,"RestingECG"]=0
  elif dataframe.at[x,"RestingECG"]=="ST":
    dataframe.at[x,"RestingECG"]=1
  elif dataframe.at[x,"RestingECG"]=="LVH":
    dataframe.at[x,"RestingECG"]=2

In [13]:
dataframe["ExerciseAngina"].unique()

array(['N', 'Y'], dtype=object)

In [14]:
for x in range(len(dataframe)):
  dataframe.at[x,"ExerciseAngina"]=1 if dataframe.at[x,"ExerciseAngina"]=="Y" else 0

In [16]:
dataframe["ST_Slope"].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [17]:
for x in range(len(dataframe)):
  if dataframe.at[x,"ST_Slope"]=="Up":
    dataframe.at[x,"ST_Slope"]=0
  elif dataframe.at[x,"ST_Slope"]=="Flat":
    dataframe.at[x,"ST_Slope"]=1
  elif dataframe.at[x,"ST_Slope"]=="Down":
    dataframe.at[x,"ST_Slope"]=2

In [18]:
dataframe

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,0,140,289,0,0,172,0,0.0,0,0
1,49,0,1,160,180,0,0,156,0,1.0,1,1
2,37,1,0,130,283,0,1,98,0,0.0,0,0
3,48,0,2,138,214,0,0,108,1,1.5,1,1
4,54,1,1,150,195,0,0,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,3,110,264,0,0,132,0,1.2,1,1
914,68,1,2,144,193,1,0,141,0,3.4,1,1
915,57,1,2,130,131,0,0,115,1,1.2,1,1
916,57,0,0,130,236,0,2,174,0,0.0,1,1


In [19]:
X=dataframe.drop(columns="HeartDisease",axis=1)

In [20]:
Y=dataframe["HeartDisease"]

In [25]:
X.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,1,0,140,289,0,0,172,0,0.0,0
1,49,0,1,160,180,0,0,156,0,1.0,1
2,37,1,0,130,283,0,1,98,0,0.0,0
3,48,0,2,138,214,0,0,108,1,1.5,1
4,54,1,1,150,195,0,0,122,0,0.0,0


In [26]:
Y.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [27]:
(x_train,x_test,y_train,y_test)=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

**Maximum score obtained from KNeighborsClassifier**

In [78]:
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.pipeline import Pipeline
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)
model = Pipeline([('nca', nca), ('knn', knn)])
model.fit(x_train, y_train)
print(model.score(x_test, y_test))

0.6141304347826086


**Maximum score obtained from Support Vector Machine**

In [74]:
from sklearn import svm
model= svm.SVC() 
model.fit(x_train, y_train)
prediction=model.predict(x_test)
score=accuracy_score(prediction,y_test)*100
print(score)

72.28260869565217


**Maximum score obtained from GuassianNB**

In [48]:
from sklearn.naive_bayes import GaussianNB
model= GaussianNB() 
model.fit(x_train, y_train)
prediction=model.predict(x_test)
score=accuracy_score(prediction,y_test)*100
print(score)

83.69565217391305


**Maximum score obtained from Bayesian Ridge Classification**

In [75]:
from sklearn import linear_model
model = linear_model.BayesianRidge()
model.fit(x_train, y_train)
prediction=model.predict(x_test)
prediction=[round(x) for x in prediction]
score=accuracy_score(prediction,y_test)*100
print(score)

84.23913043478261


**Maximum score obtained from Logistic Regression**

In [53]:
from sklearn.linear_model import LogisticRegression
model= LogisticRegression(solver='lbfgs', max_iter=1000) 
model.fit(x_train, y_train)
prediction=model.predict(x_test)
score=accuracy_score(prediction,y_test)*100
print(score)

84.23913043478261


**Maximum score obtained from Random forest classifier**

In [41]:
max=0
estimators=0
from sklearn.ensemble import RandomForestClassifier  
for x in range(1,100):
  model= RandomForestClassifier(n_estimators= x, criterion="entropy")  
  model.fit(x_train, y_train)
  prediction=model.predict(x_test)
  score=accuracy_score(prediction,y_test)*100
  if score>max:
    estimators=x
    max=score

In [42]:
print(max)
print(estimators)

91.30434782608695
26
